In [4]:

import os, sys, pandas as pd, numpy as np
#from pandasql import sqldf as sql 
#pd.set_option("max_rows", 100)
pd.options.display.max_rows = 10
pd.options.display.max_rows
import plotly as pyplot 
import plotly.io as pio
from datetime import date, datetime



print(f'np {np.__version__} pd {pd.__version__} pyplot {pyplot.__version__}' )

sys.path.append('./')
sys.path.append('../')
sys.path.append('../../')

import wf_lib2.crm_definitions as crm_definitions
from wf_lib2.crm_definitions import * 
from wf_lib2.views.plots import ViewModel2 
from wf_lib2.models.crm_p import CRMPSingle, CRMP 
from wf_lib2.models.crm_factory import crm_factory 

from wf_lib2.data.crm_dataset import CRMDataset
from wf_lib2.data.crm_pattern import CRMPattern



np 1.23.5 pd 1.3.5 pyplot 5.22.0


ImportError: cannot import name 'root_mean_squared_error' from 'sklearn.metrics' (c:\Users\xteijeiro\AppData\Local\miniconda3\envs\INNOVATIONPY39\lib\site-packages\sklearn\metrics\__init__.py)

balancing string....
full balancing


### Generate some synthetic data 

In [3]:
# what is the shape of the data expected? 
# a stack of patterns, i.e. the data as in the single zone but one more column

# synthetic data 

multi_zone_pattern = CRMPattern().generate_default_multiwell_pattern()
multi_zone_pattern.water_injection.head(5)

water_injection = [ ] 
liquid_production = [ ] 


for zone_index in [1,2,3]:
    w = multi_zone_pattern.water_injection.copy()
    zone_name = 'Zone'+str(zone_index)
    w['Zone'] = zone_name
    water_injection.append( w )

    l = multi_zone_pattern.liquid_production.copy()
    l['Zone'] = zone_name
    liquid_production.append( l )

water_injection = pd.concat( water_injection,axis=0)
liquid_production = pd.concat( liquid_production,axis=0)

multi_zone_pattern[WATER_INJECTION_KEYS[0]] = water_injection.copy()
multi_zone_pattern[LIQUID_PRODUCTION_KEYS[0]] = liquid_production.copy()

display('water injection', multi_zone_pattern.water_injection.sample(5))
display('liquid production', multi_zone_pattern.liquid_production.sample(5))


NameError: name 'CRMPattern' is not defined

In [6]:

args = dict(
    collapse_zones = False,
    crm_model = 'crm_p',

    dates= ['1950-09-23', '2200-01-28'],
    parameters = dict(tau = 10.0,  bounds= (1, 50), init_value= 5.0),
    taup = dict(bounds=(1, 50), init_value=5.0),
    dt= 1.0,
    max_running_time= 1000.0,
    optimizer= {'maxiter': 1000, 'name': 'Nelder-Mead', 'tolerance': 0.001},
    pre_optimizer= {'name': 'TNC'},
    integrated= False
) 


 
class CRM_Independent_Multizone_Allocation_Workflow: # which is not the same as CRM_Multizone to be developed

    '''
    This workflow will run independent CRM models for a number of zones in the dataset passed to the run method. 
    The run method will store internally a copy of the results per zone in a dictiopnary, where the zone name is the key 
    The results stored per zone are identical to those produced by a multi well CRM model (either p, ip or tank)

    The input data for the run method is like the CRM multi-well (a pattern) but the last column (named ZONE, zone, Zone, etc...) is the Zone.
    In this version, it is simply a bunch of patterns stacked vertically. 

    This is an example:

    DATE           Inj0	      Inj1	       Inj2	      Inj3      Zone
    2022-06-17	399.999169	243.389524	335.662023	156.936566	Zone1
    2022-08-30	352.386584	156.452475	253.726807	194.981547	Zone2
    2022-07-29	290.848856	171.812376	100.121040	212.978104	Zone2
    2022-01-25	354.720018	225.495763	196.571086	230.001693	Zone1
    2022-07-28	290.850903	171.815419	100.120641	212.978397	Zone1
    
    (for liquid production, oil production and water production, just as the CRM patterns stacked vertically)
    DATE  Producer1	Producer2	Producer3	Zone			
    2022-10-21	463.829868	231.914934	115.957467	Zone1
    2023-01-01	484.422748	242.211374	121.105687	Zone2
    2022-12-02	431.001791	215.500895	107.750448	Zone1
    2023-01-07	482.796586	241.398293	120.699146	Zone1
    2022-05-22	444.721293	222.360646	111.180323	Zone1
    '''


    def __init__(self):

        self.clear()
        

    def _init_state(self, args)->dict:
        
        args['counter'] = 0 
        args['start_time'] = datetime.now() 
        args['elapsed_time'] = 0 
     
        self._state = { }
        self._state.update( args )
        
        return self._state 
    
       
    def clear( self ):
        self._models = None       #one multi-well model per zone 
        self._data   = None 
        self._args   = None 
        self._state  = None 
        self._models = None 
        self._zone_model_map = None 


    def _collapse_zones( self, data:pd.DataFrame )->pd.DataFrame:
        raise Exception('Known exception: Not implemented yet!')
        return data
    

    def get_default_parameters( self )->str:
        default_args = dict(collapse_zones = False, crm_model = 'crm_p')
        return default_args


    # private 
    def _process_args(self, input_args: dict = None )->dict:

   
        args = self.get_default_parameters()
        if input_args is not None:
           args.update( input_args if input_args is not None else dict() )

            #
            # some other valiodation goes here 
            # if an error, set args to None 
        return args
        
    
    def fit_preprocess( self, data = None, input_args: dict = None ) -> tuple[str,Exception]:
        print('Note that in this baby version, we use the same default args for each zone and each pattern in the zone')
 
 
        self.clear() 
        args, _data = None, None 

        args = self._process_args( input_args )  #just one set, like single-zone 

        print('collapsing zones ?', args.get( 'collapse_zones', False ) )
        _data  = self._collapse_zones( data ) if args.get( 'collapse_zones', False ) else data.copy()  



        self._models, crm_model_name_typename = [], args['crm_model'] 
        self._zone_model_map = dict()

        try:

            if 'Zone' not in data.water_injection:
                if data.water_injection is not None: data.water_injection['Zone']   = 'Unique'
                if data.liquid_production is not None: data.liquid_production['Zone'] = 'Unique'
                if data.oil_production is not None: data.oil_production['Zone']    = 'Unique'
                if data.water_production is not None: data.water_production['Zone']  = 'Unique'

            zone_names = data.water_injection['Zone'].unique()
            for n,zone_name in enumerate(zone_names):

                print( '----pre-fitting zone {}'.format( zone_name))
                pattern_zone = CRMPattern()
                water_injection_zone = data.water_injection[ data.water_injection['Zone'] == zone_name ]
                liquid_production_zone = data.liquid_production[ data.liquid_production['Zone']  == zone_name ]


                pattern_zone[WATER_INJECTION_KEYS[0]] = water_injection_zone.drop( ['Zone'], axis = 1)
                pattern_zone[LIQUID_PRODUCTION_KEYS[0]] = liquid_production_zone.drop( ['Zone'], axis = 1)
                pattern_zone[LOCATION_KEYS[0]] = data[LOCATION_KEYS[0]].copy()

                print('----pre-fitting the crm model for the zone')
                self._models.append( crm_factory( crm_model_name_typename )[0] )
                model_for_zone = self._models[-1:][0] 
             
                model_for_zone.fit_preprocess( pattern_zone, input_args  )
                self._zone_model_map[zone_name] = model_for_zone

        
        except Exception as e:
            print('Failed pre-fit in zone ', zone_name )
            print( model_for_zone.log )
            print(e)
            raise e 


        #only initialize state variables if there was no error 
        self._init_state( args )
        self._data = _data     #already collapsed if that was the case. 
        self._args  = args   

        return self._data, self._args   
    
 
    def fit( self, serial = False  ):
        print('Fitting....')
        
        for zone_name, model in self._zone_model_map.items():
            model.fit( serial = False  ) 
            

    def predict( self ):
        print('Generating rate curves....')
        

        results, data, models = {}, self._data, self._models
        zone_names = data.water_injection['Zone'].unique()

        for zone_name, model in self._zone_model_map.items():
            print('Curves for zone {}'.format(  zone_name ))
            model.predict() 

        
        return 'The results are returned here'
    
  
    def run(self, input_data = None, input_args: dict = None ) -> tuple[str,Exception]:
        

        input_args = { 'serial': False }
       
        try: 
            self.fit_preprocess( input_data, input_args )
            print('fit preprocess done')
        
        except Exception as e: 
            print('fit pre-process error ',e)
            raise e 
        
        try:      

            self.fit( )  

        except Exception as e: 
            print('fit error ',e)
            raise e
        
        try:      

            self.predict()  

        except Exception as e: 
            print('predict error ',e)
            raise e


        return True 

    def get_production_allocation( self, producer_wells = None, zones = None ):
        df = [] 
        for zone_name, model in workflow._zone_model_map.items():
            print( zone_name )
            d = model.prediction_result['crm'].copy()
            d['Zone'] = zone_name
            df.append( d )
        
        df = pd.concat( df,axis = 0) 

        if zones is not None:
            df = df[ df['Zone'].isin( zones ) ]


        if producer_wells is not None:
            df = df[ df['PRODUCER'].isin( producer_wells ) ]
            
       

        return df 


    def get_fit_rates( self, producer_wells = None, zones = None ):

        #FIXME: very inneficient filtering.  
        
        map_zones = self._zone_model_map
        if zones is not None: 
            map_zones = { zone:m for zone,m in self._zone_model_map.items() if zone in zones  }
        

        df = [] 
        for zone_name, model in map_zones.items():
            d = map_zones[zone_name].prediction_result['rates'].copy()
            d['Zone'] = zone_name 
            df.append( d  )
        df = pd.concat( df, axis = 0 )


        if producer_wells is not None:
            df= df[ df['NAME'].isin(producer_wells) ]

        return df 


try: 
    workflow  = CRM_Independent_Multizone_Allocation_Workflow()
    %time success  = workflow.run(  input_data = multi_zone_pattern, input_args = args ) 
    print('success',success  )
    
except Exception as e:
    print('success', False )
    print( str(e) )
 



Note that in this baby version, we use the same default args for each zone and each pattern in the zone
collapsing zones ? False
----pre-fitting zone Zone1
----pre-fitting the crm model for the zone
----pre-fitting zone Zone2
----pre-fitting the crm model for the zone
----pre-fitting zone Zone3
----pre-fitting the crm model for the zone
fit preprocess done
Fitting....
Generating rate curves....
Curves for zone Zone1
Curves for zone Zone2
Curves for zone Zone3
CPU times: total: 188 ms
Wall time: 2.25 s
success True


In [7]:
#3 miliseconds 
workflow.get_fit_rates( producer_wells = ['Producer1'], zones= ['Zone2','Zone3']).sample(10)


,LIQUID_VOLUME_sim,CUM_WATER_INJECTED_sim,LIQUID_VOLUME,Lo,NAME,TRAIN,ID,prod_id,pattern_id,Zone
DATE,,,,,,,,,,
2022-05-08,432.597404,40723.549338,432.379939,3.175091e-01,Producer1,True,104,0,0,Zone2
2022-11-13,476.539020,126104.798799,476.734664,2.239721e-07,Producer1,True,293,0,0,Zone2
2022-08-26,425.869227,90490.411085,426.510128,8.346067e-05,Producer1,True,214,0,0,Zone2
2022-10-03,444.755943,107309.212661,445.166130,4.837933e-06,Producer1,True,252,0,0,Zone2
2022-08-24,423.853380,89639.649645,424.504409,9.695669e-05,Producer1,True,212,0,0,Zone3
2022-12-29,483.359500,146992.381483,483.515131,7.128338e-09,Producer1,True,339,0,0,Zone2
2022-09-06,437.849645,95244.363816,438.141567,3.659775e-05,Producer1,True,225,0,0,Zone3
2022-05-27,457.963571,49143.511878,457.697442,7.644431e-02,Producer1,True,123,0,0,Zone3
2022-06-27,479.434750,63763.040632,479.829923,7.487878e-03,Producer1,True,154,0,0,Zone3


In [1]:
workflow.get_production_allocation( zones = ['Zone1', 'Zone2', 'Zone3'], producer_wells = ['Producer1', 'Producer2', 'Producer3'])

NameError: name 'workflow' is not defined

,INJECTOR,PRODUCER,allocation,tau,taup,productivity,Lo,MODEL,ID,pattern_id,prod_id
0,Inj0,Producer1,0.605723,16.200724,13.343221,0.0,1.002976,CRMP,0,0,0
1,Inj1,Producer1,0.494262,16.200724,13.343221,0.0,1.002976,CRMP,1,0,0
2,Inj2,Producer1,0.398508,16.200724,13.343221,0.0,1.002976,CRMP,2,0,0
3,Inj3,Producer1,0.308608,16.200724,13.343221,0.0,1.002976,CRMP,3,0,0
